# Pumpfun Tools

## Herramientas de PumpFun Price Monitor

Este notebook contiene herramientas especializadas para monitorear precios en tiempo real de tokens de Pump.fun usando WebSocket.

### Importaciones y Configuración

In [ ]:
from solana_manager import SolanaWalletManager, SolanaAccountInfo
from pumpfun import PumpFunPriceMonitor, PumpFunPriceFetcher, PumpFunTrader
import asyncio
import time
from datetime import datetime

In [ ]:
# Configuración básica
NETWORK = "mainnet-beta"
RPC_URL = "https://api.mainnet-beta.solana.com"
WALLET_FILE = "wallets/wallet_pumpfun.json"

# Inicializar componentes
wallet_manager = SolanaWalletManager(network=NETWORK, rpc_url=RPC_URL)
if not wallet_manager.load_wallet(WALLET_FILE):
    raise Exception("Error al cargar la wallet")

# Inicializar herramientas de PumpFun
price_monitor = PumpFunPriceMonitor()
price_fetcher = PumpFunPriceFetcher(wallet_manager)
trader = PumpFunTrader(wallet_manager)


### 1. Monitor de Token Específico (SIN ruido)

Monitorea ÚNICAMENTE el token que especifiques, sin mostrar otros eventos.

In [ ]:
# Configurar token para monitorear
TOKEN_TO_MONITOR = "4yyuDzaxZdXbCsMeLLbHmoVYJoquq6xo1LYfBaZrpump"  # Ejemplo: token TML

# Configurar alertas de precio (opcional)
price_alerts = {
    'above': 0.000005,  # Alerta si precio sube por encima de este valor
    'below': 0.000003   # Alerta si precio baja por debajo de este valor
}

print(f"🎯 Token a monitorear: {TOKEN_TO_MONITOR}")
print(f"📊 Alertas configuradas:")
print(f"   ⬆️ Alerta HIGH: {price_alerts['above']:.10f} SOL")
print(f"   ⬇️ Alerta LOW: {price_alerts['below']:.10f} SOL")


In [ ]:
# 🎯 NUEVO MÉTODO CORREGIDO: Solo monitorea TU token (SIN otros eventos)
print("🔄 Usando método corregido que NO muestra otros tokens...")

# Usar el nuevo método que crea una conexión específica
stats_clean = price_monitor.monitor_single_token_only(
    token_address=TOKEN_TO_MONITOR,
    duration_minutes=1,  # 1 minuto para prueba
    show_trades=True,
    price_alerts=price_alerts
)

print(f"\n✅ Monitoreo limpio completado!")
print(f"📈 Trades de TU token: {stats_clean['trade_count']}")
print(f"💰 Volumen de TU token: {stats_clean['total_volume']:.6f} SOL")


### 2. Monitor de Nuevos Tokens

Detecta nuevos tokens creados en Pump.fun en tiempo real.

In [ ]:
# Monitorear nuevos tokens por 3 minutos
new_tokens = price_monitor.monitor_new_tokens(
    duration_minutes=1,
    auto_subscribe=True,  # Auto-suscribirse a trades de nuevos tokens
    max_tokens=3         # Máximo 3 tokens para no saturar
)

print(f"\n🆕 Nuevos tokens detectados: {len(new_tokens)}")
for i, token in enumerate(new_tokens, 1):
    print(f"{i}. {token['mint'][:8]}...")
    print(f"   💰 Market Cap: {token['market_cap']:.2f} SOL")
    print(f"   👤 Creador: {token['creator']}...")
    print(f"   ⏰ Hora: {token['timestamp'].strftime('%H:%M:%S')}")


### 3. Análisis de Bonding Curve

Obtiene información detallada de la bonding curve de un token.

In [ ]:
# Analizar bonding curve de un token
token_price = price_fetcher.get_token_price(TOKEN_TO_MONITOR)

if token_price:
    print("📊 ANÁLISIS DE BONDING CURVE")
    print("=" * 50)
    print(f"🎯 Token: {token_price.token_address}")
    print(f"💰 Precio SOL: {token_price.price_sol:.12f}")
    print(f"💵 Precio USD: ${token_price.price_usd:.12f}")
    print(f"📈 Market Cap: ${token_price.market_cap_usd:,.2f}")
    print(f"📊 Progreso Bonding: {token_price.bonding_progress:.2f}%")
    print(f"⏰ Timestamp: {token_price.timestamp}")
    
    # Detalles de la curve
    curve = token_price.curve_state
    print(f"\n🔍 ESTADO DE LA CURVE:")
    print(f"   🪙 Reservas virtuales token: {curve.virtual_token_reserves:,}")
    print(f"   💎 Reservas virtuales SOL: {curve.virtual_sol_reserves:,}")
    print(f"   🪙 Reservas reales token: {curve.real_token_reserves:,}")
    print(f"   💎 Reservas reales SOL: {curve.real_sol_reserves:,}")
    print(f"   📊 Supply total: {curve.token_total_supply:,}")
    print(f"   ✅ Completa: {'Sí' if curve.complete else 'No'}")
else:
    print("❌ No se pudo obtener información del token")


### 4. Limpieza y Cierre

Funciones para limpiar conexiones y cerrar el monitor correctamente.

In [ ]:
# Función para limpiar y cerrar conexiones
def cleanup_monitor():
    """
    Limpia y cierra todas las conexiones del monitor
    """
    print("🧹 Limpiando conexiones...")
    
    try:
        if price_monitor.is_running:
            price_monitor.stop_monitoring()
            print("✅ Monitor detenido")
        
        # Limpiar suscripciones
        price_monitor.subscribed_tokens.clear()
        price_monitor.subscribed_accounts.clear()
        
        print("✅ Suscripciones limpiadas")
        print("🔌 Monitor listo para nueva sesión")
        
    except Exception as e:
        print(f"⚠️ Error durante limpieza: {e}")

# Ejecutar limpieza al final (descomenta si necesitas)
cleanup_monitor()
